In [3]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 999)
import seaborn as sns

os.chdir(r'\\mbip\medicarepBI\Projects\COE\Retention\Deliverables\Refreshed Models AEP\2019_scores'.replace("\\",'/'))

SyntaxError: unexpected character after line continuation character (<ipython-input-3-63b696656c3a>, line 10)

In [1]:
scores=pd.read_csv('//mbip/medicarepBI/Projects/COE/Retention/Deliverables/Refreshed Models AEP/retention_Refreshed_aep_models_summary_D14Nov2019_updated_v1.csv'
                   ,engine='c',
                  dtype={'SRC_MEMBER_ID':str})
scores=scores.iloc[:,1:]
scores['DECILE_CATEGORY'] = np.where(((scores.Decile >= 1) & (scores.Decile <= 3)),'HIGH DECILE',
                                             np.where(((scores.Decile >= 4) & (scores.Decile <= 7)),'MEDIUM DECILE',
                                                     np.where(((scores.Decile >= 8) & (scores.Decile <= 10)),'LOW DECILE','NA')
                                                     ))
scores.set_index('SRC_MEMBER_ID',inplace=True)

scores.head(2)

NameError: name 'pd' is not defined

In [5]:
#reading the churnbase
base2019= pd.read_csv('//mbip/medicarepBI/Projects/COE/Retention/Deliverables/Refreshed Models AEP/churnbase_merge_hpd_and_planbenefit_RY2019_14NOV2019.csv',
                      usecols=['SRC_MEMBER_ID','NEWMAP_MARKET','SNP_IND']
                     ).query('SNP_IND =="N"')

base2019.SRC_MEMBER_ID = base2019.SRC_MEMBER_ID.astype(str)
base2019.set_index('SRC_MEMBER_ID',inplace=True)

print(base2019.shape)
base2019.head(1)

(1415197, 2)


,SNP_IND,NEWMAP_MARKET
SRC_MEMBER_ID,,
MEBQQQDH,N,Mid South


In [10]:
sales_Data=pd.read_csv('//mbip/medicarepBI/Projects/COE/Retention/Data/Ting/Data Dump 20191115/YTD_MA_2019_DataDump.csv'
                       ,usecols=['NEW_MARKET', 'MA_Market_PDP_Product', 'LIS_Flag', 'Enroll_Status',
                                  'Member_ID', 'EFF_Date', 'Term_Date', 'Contract_Number', 'PBP', 'FIRST_NAME',
                                  'LAST_NAME', 'MIDDLE_INITIAL', 'Group_Status', 'CMS_County', 'CMS_State', 'MA_Region',
                                  'NEW_REGION', 'MA_Territory', 'Age', 'Gender', 'Plan_Name', 'Plan_Type', 'Address1',
                                  'Address2', 'City', 'ZIP', 'FIPS', 'Writing_Agent_ID', 'Writing_Agent_Name', 'TOH_Name',
                                  'TOH_ID', 'Sales_Channel', 'Historic_Sales_Channel', 'Recruiter_Name', 'Recruiter_ID',
                                  'Mbr_Mths', 'BEQ_Prior_Organization'],engine='c',encoding='ISO-8859-1')

sales_Data.Member_ID=sales_Data.Member_ID.astype(str)
sales_Data.Member_ID=sales_Data.Member_ID.str.split('*').str.join('').str.strip()
sales_Data.Member_ID = sales_Data.Member_ID.str.strip()
sales_Data.rename(columns = {'Member_ID':'SRC_MEMBER_ID'},inplace=True)
sales_Data.SRC_MEMBER_ID = sales_Data.SRC_MEMBER_ID.astype(str)
sales_Data.head(1)

C:\Users\N275989\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (48,49,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NEW_MARKET,MA_Market_PDP_Product,LIS_Flag,Enroll_Status,SRC_MEMBER_ID,EFF_Date,Term_Date,Contract_Number,PBP,FIRST_NAME,LAST_NAME,MIDDLE_INITIAL,Group_Status,CMS_County,CMS_State,MA_Region,NEW_REGION,MA_Territory,Age,Gender,Plan_Name,Plan_Type,Address1,Address2,City,ZIP,FIPS,Writing_Agent_ID,Writing_Agent_Name,TOH_Name,TOH_ID,Sales_Channel,Historic_Sales_Channel,Recruiter_Name,Recruiter_ID,Mbr_Mths,BEQ_Prior_Organization
0,OHKY,OHKY,NaN,Active,MEBSD1SS,01APR2019,31DEC9999,H5521,89,KATHY,WHITTECAR,D,Individual,FRANKLIN,OH,MIDATLANTIC,MIDATLANTIC,OH_COLUMBUS,68,F,Aetna Medicare Value Plan (PPO),PPO,92 S SOUTHAMPTON AVE,NaN,COLUMBUS,432041934,39049,192953,"Quick, Kerry",Integrity Marketing Group,283186.0,NDP,Broker_Related,"Seniority Benefit Group, Inc.",177437.0,8,NaN


In [14]:
sales_Data.drop_duplicates(inplace=True)
sales_Data.shape,sales_Data.SRC_MEMBER_ID.nunique()

((1394685, 37), 1394674)

In [15]:
sales_Data['EFF_Date'] = pd.to_datetime(sales_Data['EFF_Date'], format='%d%b%Y',errors='coerce')
sales_Data['BROKER_FLAG'] = np.where(sales_Data.Sales_Channel.isin(['EMPGROUPEXCHANGE','NDP','STRATEGIC','DIRECT','CAREFREESOLUTIONS']),'BROKER','NON-BROKER')
sales_Data['RN_FLAG']= sales_Data.groupby("SRC_MEMBER_ID")["EFF_Date"].rank(ascending=False)
sales_Data = sales_Data[sales_Data['RN_FLAG'] == 1]
sales_Data.set_index('SRC_MEMBER_ID',inplace=True)
sales_Data.shape,sales_Data.index.nunique()

((1394673, 38), 1394673)

In [16]:
final = base2019.join(scores,how='inner')
print(final.shape)
final = final.join(sales_Data,how='inner')
print(final.shape,final.index.nunique())
# print(set(final.columns)-set(base.columns))
final.head(1)

(1415197, 6)
(1308480, 44) 1308480


,SNP_IND,NEWMAP_MARKET,INDIVIDUAL_ID,PROPENSITY,Decile,DECILE_CATEGORY,NEW_MARKET,MA_Market_PDP_Product,LIS_Flag,Enroll_Status,EFF_Date,Term_Date,Contract_Number,PBP,FIRST_NAME,LAST_NAME,MIDDLE_INITIAL,Group_Status,CMS_County,CMS_State,MA_Region,NEW_REGION,MA_Territory,Age,Gender,Plan_Name,Plan_Type,Address1,Address2,City,ZIP,FIPS,Writing_Agent_ID,Writing_Agent_Name,TOH_Name,TOH_ID,Sales_Channel,Historic_Sales_Channel,Recruiter_Name,Recruiter_ID,Mbr_Mths,BEQ_Prior_Organization,BROKER_FLAG,RN_FLAG
SRC_MEMBER_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MEBQQQDH,N,Mid South,852860,0.464857,3,HIGH DECILE,MIDSOUTH,MIDSOUTH,1.0,Active,2019-01-01,31DEC9999,H5521,236,MERITTA,BLANTON,A,Individual,RUTHERFORD,NC,SOUTHEAST,SOUTHEAST,NC_WESTERN,64,F,Aetna Medicare Premier Plan (PPO),PPO,3135 HOLLIS RD,NaN,ELLENBORO,280409774,37161,184523,"Bielamowicz, Thomas","Agent Pipeline, Inc.",183916.0,NDP,Broker_Related,"Agent Pipeline, Inc.",183916.0,11,Humana Inc.,BROKER,1.0


In [26]:
final.drop(columns=['RN_FLAG','SNP_IND'],axis=1).to_csv('Score_2019_11nov.csv')

In [25]:
print(set(final.columns)-set(base.columns))

{'RN_FLAG', 'BROKER_FLAG', 'SNP_IND', 'NEWMAP_MARKET', 'DECILE_CATEGORY'}


In [2]:
base = pd.read_csv('Score_2019_11nov.csv',
                   engine='c',
                  dtype={'SRC_MEMBER_ID':str}
                  )
#base.drop(base.columns[0],axis=1,inplace=True)
# base.head()
# base.columns.to_list()
print(base.shape)
base.head(1)

C:\Users\N275989\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1308480, 43)


,SRC_MEMBER_ID,NEWMAP_MARKET,INDIVIDUAL_ID,PROPENSITY,Decile,DECILE_CATEGORY,NEW_MARKET,MA_Market_PDP_Product,LIS_Flag,Enroll_Status,EFF_Date,Term_Date,Contract_Number,PBP,FIRST_NAME,LAST_NAME,MIDDLE_INITIAL,Group_Status,CMS_County,CMS_State,MA_Region,NEW_REGION,MA_Territory,Age,Gender,Plan_Name,Plan_Type,Address1,Address2,City,ZIP,FIPS,Writing_Agent_ID,Writing_Agent_Name,TOH_Name,TOH_ID,Sales_Channel,Historic_Sales_Channel,Recruiter_Name,Recruiter_ID,Mbr_Mths,BEQ_Prior_Organization,BROKER_FLAG
0,MEBQQQDH,Mid South,852860,0.464857,3,HIGH DECILE,MIDSOUTH,MIDSOUTH,1.0,Active,2019-01-01,31DEC9999,H5521,236,MERITTA,BLANTON,A,Individual,RUTHERFORD,NC,SOUTHEAST,SOUTHEAST,NC_WESTERN,64,F,Aetna Medicare Premier Plan (PPO),PPO,3135 HOLLIS RD,NaN,ELLENBORO,280409774,37161,184523.0,"Bielamowicz, Thomas","Agent Pipeline, Inc.",183916.0,NDP,Broker_Related,"Agent Pipeline, Inc.",183916.0,11,Humana Inc.,BROKER


In [3]:
base.columns[base.columns.str.contains('ID')]

Index(['SRC_MEMBER_ID', 'INDIVIDUAL_ID', 'MIDDLE_INITIAL', 'Writing_Agent_ID',
       'TOH_ID', 'Recruiter_ID'],
      dtype='object')

In [4]:
base.NEWMAP_MARKET.value_counts()

Keystone         198490
Heartland        168172
Mid South        127308
Deep South       116231
Texas            104898
Great Lakes       89801
New York          81648
Florida           72858
Ohio              64447
New Jersey        61035
New England       59185
Midlands          57762
Northwest         23932
North Capitol     21470
California        18436
Arizona           17373
Nevada            12478
Utah               9564
Colorado           3392
Name: NEWMAP_MARKET, dtype: int64

In [6]:
base.NEWMAP_MARKET = base.NEWMAP_MARKET.str.strip().str.upper()
base[base.NEWMAP_MARKET.isin(['ARIZONA'])].shape

(17373, 43)

In [8]:
data = base[base.NEWMAP_MARKET.isin(['ARIZONA'])]
print(data.shape)

# data = data[data.SNP!='Y']
print(data.shape)
data.head(2)

(17373, 43)
(17373, 43)


,SRC_MEMBER_ID,NEWMAP_MARKET,INDIVIDUAL_ID,PROPENSITY,Decile,DECILE_CATEGORY,NEW_MARKET,MA_Market_PDP_Product,LIS_Flag,Enroll_Status,EFF_Date,Term_Date,Contract_Number,PBP,FIRST_NAME,LAST_NAME,MIDDLE_INITIAL,Group_Status,CMS_County,CMS_State,MA_Region,NEW_REGION,MA_Territory,Age,Gender,Plan_Name,Plan_Type,Address1,Address2,City,ZIP,FIPS,Writing_Agent_ID,Writing_Agent_Name,TOH_Name,TOH_ID,Sales_Channel,Historic_Sales_Channel,Recruiter_Name,Recruiter_ID,Mbr_Mths,BEQ_Prior_Organization,BROKER_FLAG
73,MEBRXDHM,ARIZONA,3288422,0.109514,8,LOW DECILE,ARIZONA,ARIZONA,NaN,Active,2019-01-01,31DEC9999,H3931,129,TERRY,THOMPSON,N,Individual,PINAL,AZ,NORTHCENTRAL,NORTHCENTRAL,AZ_CENTRAL,69,M,Aetna Medicare Platinum Plan (HMO),HMO,1136 E SHEPHERDS WAY,NaN,CASA GRANDE,851222956,4021,NaN,NaN,NaN,NaN,CMSOEC,CMSOEC,NaN,NaN,11,Aetna Inc.,NON-BROKER
113,MEBQXMBD,ARIZONA,4614214,0.449735,2,HIGH DECILE,ARIZONA,ARIZONA,1.0,Active,2019-01-01,31DEC9999,H3931,131,JERRI,GUNTER,S,Individual,GILA,AZ,NORTHCENTRAL,NORTHCENTRAL,AZ_CENTRAL,79,F,Aetna Medicare Platinum Plan (HMO),HMO,1304 N BEELINE HWY,SP 80,PAYSON,85541,4007,219461.0,"Ellingson, Nylene","Agent Pipeline, Inc.",183916.0,NDP,Broker_Related,Empire Financial Firm,169368.0,11,NaN,BROKER


In [10]:
data.Sales_Channel.fillna('No_sales_Channel',inplace=True)
data.Sales_Channel = data.Sales_Channel.str.strip().str.upper()

In [8]:
#Create Broker Flag
# data['BROKER_FLAG'] = np.where(data.Sales_Channel.isin(['EMPGROUPEXCHANGE','NDP','STRATEGIC','DIRECT'
#                                                                                   ,'CAREFREE'
# #                                                         ,'HEALTHSPIRE'
# #                                                         ,'MEDICARE TRANSITION SERVICES'
#                                                        ]),'BROKER','NON-BROKER')


In [13]:
ytd= pd.read_csv(r'\\mbip\medicarepBI\Projects\COE\Retention\Data\Ting\Data Dump 20191115\YTD_MA_2019_DataDump.csv'.replace('\\','/')
            ,skipinitialspace=True
            ,engine='c'
                 ,encoding='ISO-8859-1'
                 ,usecols=['Member_ID','MEM_Phone']
                 ,dtype={'Member_ID':str}
            ).rename(columns={'Member_ID':'SRC_MEMBER_ID'})                   

ytd.SRC_MEMBER_ID = ytd.SRC_MEMBER_ID.str.split('*').str.join('')
ytd.head()

C:\Users\N275989\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SRC_MEMBER_ID,MEM_Phone
0,MEBSD1SS,6144320603
1,MEBLMFRN,7704751256
2,MEBKX06X,7704030562
3,MEBRGSXN,2814606311
4,MEBN70BZ,8047955747


In [14]:
len(set(ytd.SRC_MEMBER_ID)& set(data.SRC_MEMBER_ID))

17373

In [15]:
data.head(1)

,SRC_MEMBER_ID,NEWMAP_MARKET,INDIVIDUAL_ID,PROPENSITY,Decile,DECILE_CATEGORY,NEW_MARKET,MA_Market_PDP_Product,LIS_Flag,Enroll_Status,EFF_Date,Term_Date,Contract_Number,PBP,FIRST_NAME,LAST_NAME,MIDDLE_INITIAL,Group_Status,CMS_County,CMS_State,MA_Region,NEW_REGION,MA_Territory,Age,Gender,Plan_Name,Plan_Type,Address1,Address2,City,ZIP,FIPS,Writing_Agent_ID,Writing_Agent_Name,TOH_Name,TOH_ID,Sales_Channel,Historic_Sales_Channel,Recruiter_Name,Recruiter_ID,Mbr_Mths,BEQ_Prior_Organization,BROKER_FLAG
73,MEBRXDHM,ARIZONA,3288422,0.109514,8,LOW DECILE,ARIZONA,ARIZONA,NaN,Active,2019-01-01,31DEC9999,H3931,129,TERRY,THOMPSON,N,Individual,PINAL,AZ,NORTHCENTRAL,NORTHCENTRAL,AZ_CENTRAL,69,M,Aetna Medicare Platinum Plan (HMO),HMO,1136 E SHEPHERDS WAY,NaN,CASA GRANDE,851222956,4021,NaN,NaN,NaN,NaN,CMSOEC,CMSOEC,NaN,NaN,11,Aetna Inc.,NON-BROKER


In [23]:
temp =data[['SRC_MEMBER_ID','FIRST_NAME','LAST_NAME','MIDDLE_INITIAL','Age'
            ,'Gender','LIS_Flag','Contract_Number','PBP','CMS_County','TOH_Name'
            ,'Recruiter_Name','Writing_Agent_Name','BROKER_FLAG','Mbr_Mths','PROPENSITY'
            ,'DECILE_CATEGORY']].set_index('SRC_MEMBER_ID').join(ytd.set_index('SRC_MEMBER_ID'),how='left')

temp.reset_index(inplace=True)
temp.rename(columns={'SRC_MEMBER_ID':'Source Member ID','BROKER_FLAG':'Sales Channel'
                     ,'PROPENSITY':'Lapse Risk Propensity Score','MEM_Phone':'Member Phone'
                    ,'DECILE_CATEGORY':'Lapse_Decile'},inplace=True)


In [24]:
temp[['Source Member ID', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_INITIAL','Member Phone', 'Age',
       'Gender', 'LIS_Flag', 'Contract_Number', 'PBP', 'CMS_County',
       'TOH_Name', 'Recruiter_Name', 'Writing_Agent_Name', 'Sales Channel',
       'Mbr_Mths', 'Lapse Risk Propensity Score', 'Lapse_Decile',
       ]].head(2)

,Source Member ID,FIRST_NAME,LAST_NAME,MIDDLE_INITIAL,Member Phone,Age,Gender,LIS_Flag,Contract_Number,PBP,CMS_County,TOH_Name,Recruiter_Name,Writing_Agent_Name,Sales Channel,Mbr_Mths,Lapse Risk Propensity Score,Lapse_Decile
0,80340856601,EUGENE,CRUMP,NaN,678-548-7458,79,M,NaN,H1608,28,MARICOPA,ConnexionPoint Non-Licensed Agent,NaN,ConnexionPoint Non-Licensed Agent,NON-BROKER,47,0.899646,HIGH DECILE
1,80461693301,JOANNE,BUSH,E,480-438-4225,73,F,3.0,H3959,32,ERIE,Premier Senior Marketing,"Hessinger, Randall","SMITH, NICOLE",BROKER,35,0.311387,MEDIUM DECILE


In [25]:
# sample.to_csv(r'\\mbip\medicarepBI\Projects\COE\Retention\Deliverables\Refreshed Models AEP\2019_scores\ARIZONA\Refreshed_model_score_ARIZONA_D13DEC2019VER2.csv'.replace('\\','/'))

In [26]:
temp[['Source Member ID', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_INITIAL','Member Phone', 'Age',
       'Gender', 'LIS_Flag', 'Contract_Number', 'PBP', 'CMS_County',
       'TOH_Name', 'Recruiter_Name', 'Writing_Agent_Name', 'Sales Channel',
       'Mbr_Mths', 'Lapse Risk Propensity Score', 'Lapse_Decile',
       ]].to_csv(r'\\mbip\medicarepBI\Projects\COE\Retention\Deliverables\Refreshed Models AEP\2019_scores\ARIZONA\Refreshed_model_score_ARIZONA_VER2.csv'.replace('\\','/'))

In [28]:
temp1 = data.groupby(['CMS_County','DECILE_CATEGORY'])['SRC_MEMBER_ID'].nunique().unstack().fillna(0).join(
data.groupby(['CMS_County'])['SRC_MEMBER_ID'].nunique().fillna(0).to_frame('Overall Membership')).join(
    data.groupby('CMS_County')['Mbr_Mths'].mean().to_frame('Average Tenure In Months'))#.to_csv(r'\\mbip\medicarepBI\Projects\COE\Retention\Deliverables\Refreshed Models AEP\2019_scores\Florida\Refreshed_model_score_FL_county_summary.csv'.replace('\\','/'))

In [29]:
temp1.head(1)

,HIGH DECILE,LOW DECILE,MEDIUM DECILE,Overall Membership,Average Tenure In Months
CMS_County,,,,,
ADA,1.0,0.0,0.0,1,11.0


In [30]:
temp1.rename(columns={'HIGH DECILE':'High Lapse Risk Decile','LOW DECILE':'Low Lapse Risk Decile','MEDIUM DECILE':'Medium Lapse Risk Decile'},inplace=True)

In [31]:
temp1.to_csv(r'\\mbip\medicarepBI\Projects\COE\Retention\Deliverables\Refreshed Models AEP\2019_scores\ARIZONA\Refreshed_model_score_ARIZONA_county_summary_VER2.csv'.replace('\\','/'))